In [1]:
import os
import re
from pathlib import Path
from shutil import copyfile
import pandas as pd

In [2]:
data_folder = './csv_data'
total_samples = 2400

In [3]:
df_count = pd.DataFrame(columns=['topic', 'count', 'proportion_count'])

csv_dfs = {}
total_count = 0

entries = os.listdir(data_folder)
regex = re.compile('data_([a-zA-Z]*)_nodups_wobj\.csv')

for filename in entries:
    if filename.startswith('.'): continue
    matched = regex.search(filename)
    if matched:
        topic_name = matched.group(1)
        csv_path = os.path.join(data_folder, filename)
        csv_dfs[topic_name] = pd.read_csv(csv_path, index_col=0)
        row_count = len(csv_dfs[topic_name].index)
        total_count += row_count
        df_count.loc[len(df_count)] = [topic_name, row_count, -1]
        print('{:<13}{:<7}'.format(topic_name, row_count))


Political    29663  
Immigration  6854   
AsianHate    1602   
Boomer       2184   
Vaccine      2671   
Mask         20149  


In [4]:
# df_count['proportion_count'] = df_count['count'].apply(lambda x: round((x/total_count)*total_samples))
df_count['proportion_count'] = round(total_samples / len(df_count.index))

In [5]:
df_count.to_csv('./csv_data/sample_count.csv', index=False)
df_count

,topic,count,proportion_count
0,Political,29663,400
1,Immigration,6854,400
2,AsianHate,1602,400
3,Boomer,2184,400
4,Vaccine,2671,400
5,Mask,20149,400


In [6]:
df_count['proportion_count'].sum()

2400

In [7]:
df_samples = pd.DataFrame(columns=['topic', 'hashtag', 'tweet_id', 'image_path', 'sample_path', 'body_text', 'image_text'])

for _, row in df_count.iterrows():
    topic_name = row['topic']
    df_topic = csv_dfs[topic_name].sample(n=row['proportion_count'], random_state=42)
    df_topic.insert(loc=0, column='topic', value=topic_name)
    df_topic.insert(loc=4, column='sample_path', value=-1)
    df_topic.insert(loc=len(df_topic.columns), column='image_text', value=-1)
    df_samples = df_samples.append(df_topic)

df_samples.reset_index(drop=True, inplace=True)
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text
0,Political,maga,1293438333503373312,./data_Political/maga/tweets2/EfM3M8yXoAAmx8N.jpg,-1,.@realDonaldTrump and #MAGA you know your boy...,-1
1,Political,trumpvirus,1289888955568140289,./data_Political/trumpvirus/tweets1/EeabK0xWAA...,-1,@Heritage @realDonaldTrump @HvonSpakovsky @Liz...,-1
2,Political,Trump2020,1297529851587497986,./data_Political/Trump2020/tweets4/EgHAYRDWkAg...,-1,I'm getting close to 200 followers in a record...,-1
3,Political,maga,1292446789623193601,./data_Political/maga/tweets2/Ee-xgU8UEAIVBgx.jpg,-1,@w_terrence And exactly WHO do y’all give a da...,-1
4,Political,trumpvirus,1293388373223243776,./data_Political/trumpvirus/tweets2/EfMJ3QwWoA...,-1,@realDonaldTrump How canUeven ALLOW these #Fak...,-1
...,...,...,...,...,...,...,...
2395,Mask,maskup,1298350209957863424,./data_Mask/maskup/tweets4/EgSqoY-WoAQeCSK.jpg,-1,Don't forget to stay up-to-date with all the l...,-1
2396,Mask,maskup,1296596061876899842,./data_Mask/maskup/tweets3/Ef5vPcfWoAE-waN.jpg,-1,Come Help Us Celebrate Heaven’s Birthday Tonig...,-1
2397,Mask,WearAMask,1292250597593415685,./data_Mask/WearAMask/tweets2/Ee7_EdIWsAMb3KI.jpg,-1,Just a reminder to set your reminder to remind...,-1
2398,Mask,facemasks,1298568754327171072,./data_Mask/facemasks/tweets4/EgVxXGsXoAIastL.jpg,-1,#life continues\n-\n#Indian woman Sree Giri Sh...,-1


# Please only run cells below

## Copy selected samples and generate sample path

In [8]:
import os
from pathlib import Path
from shutil import copyfile
import pandas as pd

In [9]:
df_count = pd.read_csv('./csv_data/sample_count.csv')
df_samples = pd.read_csv('./csv_data/sample_data.csv')

In [10]:
def get_sample_path_and_copy(df, topic, annot_data_path):
    # if the sample_path already exists
    if df['sample_path'] != -1:
        return df['sample_path']

    # if topic is not the selected topic
    if df['sample_path'] == -1 and df['topic'] != topic:
        return -1
    
    image_name = df['image_path'].split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(df['image_path'], dst_path)
    return dst_path

In [11]:
sample_folder = './annotation_data'
Path(sample_folder).mkdir(exist_ok=True)

for _, row in df_count.iterrows():
    topic_name = row['topic']
    topic_data_path = os.path.join('./', 'data_' + topic_name)
    annot_data_path = os.path.join(sample_folder, topic_name)
    # if data_{topic_name} folder doesn't exist
    if not Path(topic_data_path).is_dir():
        print('{} not exists; pass this topic'.format(topic_data_path))
        continue

    # if the sampled data for this topic already exists
    if Path(annot_data_path).is_dir():
        print('{} already exist; pass this topic'.format(annot_data_path))
        continue

    Path(annot_data_path).mkdir()

    df_samples['sample_path'] = df_samples.apply(get_sample_path_and_copy, axis=1, topic=topic_name, annot_data_path=annot_data_path)

    print('Topic \"{}\" processed'.format(topic_name))

./data_Political not exists; pass this topic
Topic "Immigration" processed
./data_AsianHate not exists; pass this topic
./data_Boomer not exists; pass this topic
Topic "Vaccine" processed
./data_Mask not exists; pass this topic


In [12]:
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text
0,Political,maga,1293438333503373312,./data_Political/maga/tweets2/EfM3M8yXoAAmx8N.jpg,-1,.@realDonaldTrump and #MAGA you know your boy...,-1
1,Political,trumpvirus,1289888955568140289,./data_Political/trumpvirus/tweets1/EeabK0xWAA...,-1,@Heritage @realDonaldTrump @HvonSpakovsky @Liz...,-1
2,Political,Trump2020,1297529851587497986,./data_Political/Trump2020/tweets4/EgHAYRDWkAg...,-1,I'm getting close to 200 followers in a record...,-1
3,Political,maga,1292446789623193601,./data_Political/maga/tweets2/Ee-xgU8UEAIVBgx.jpg,-1,@w_terrence And exactly WHO do y’all give a da...,-1
4,Political,trumpvirus,1293388373223243776,./data_Political/trumpvirus/tweets2/EfMJ3QwWoA...,-1,@realDonaldTrump How canUeven ALLOW these #Fak...,-1
...,...,...,...,...,...,...,...
2395,Mask,maskup,1298350209957863424,./data_Mask/maskup/tweets4/EgSqoY-WoAQeCSK.jpg,-1,Don't forget to stay up-to-date with all the l...,-1
2396,Mask,maskup,1296596061876899842,./data_Mask/maskup/tweets3/Ef5vPcfWoAE-waN.jpg,-1,Come Help Us Celebrate Heaven’s Birthday Tonig...,-1
2397,Mask,WearAMask,1292250597593415685,./data_Mask/WearAMask/tweets2/Ee7_EdIWsAMb3KI.jpg,-1,Just a reminder to set your reminder to remind...,-1
2398,Mask,facemasks,1298568754327171072,./data_Mask/facemasks/tweets4/EgVxXGsXoAIastL.jpg,-1,#life continues\n-\n#Indian woman Sree Giri Sh...,-1
